In [0]:
import pandas as pd

In [0]:
flux = pd.read_csv('_Flux dataframe.csv')

In [0]:
from lightkurve.lightcurve import TessLightCurve

In [0]:
from lightkurve import TessTargetPixelFile

In [0]:
from lightkurve import search_targetpixelfile

In [0]:
tess_id = pd.read_csv('_Tess 100.csv')

In [0]:
import numpy as np
import requests
from IPython.display import display, HTML
import sys
import os
import time
import re
import json
from urllib.parse import quote as urlencode
from urllib.request import urlretrieve
import http.client as httplib
from astropy.table import Table


# def mastQuery(req):

#    """Basic mast query maker. Similar to
#    example code from astroquery documentation """

#    server='mast.stsci.edu'

#    version = ".".join(map(str, sys.version_info[:3]))

# HTTP header variables for request
#    headers = {"Content-type": "application/x-www-form-urlencoded",
#               "Accept": "text/plain",
#               "User-agent":"python-requests/"+version}

# Make request string
#    reqStr = json.dumps(req)
#    reqStr = urlencode(reqStr)

# Open https connection
#    conn = httpcli.HTTPSConnection(server)

# Run the query
#    conn.request("POST", "api/v0/invoke", "request="+reqStr, headers)

# Get and save the response
#    resp = conn.getresponse()
# Save headers
#    head = resp.getheaders()
# Save content of response
#    content = resp.read().decode('utf-8')

# Close connection
#    conn.close()

#    return head,content


def mastQuery(request):
    """Perform a MAST query.
    
        Parameters
        ----------
        request (dictionary): The MAST request json object
        
        Returns head,content where head is the response HTTP headers, and content is the returned data"""

    server = "mast.stsci.edu"

    # Grab Python Version
    version = ".".join(map(str, sys.version_info[:3]))

    # Create Http Header Variables
    headers = {
        "Content-type": "application/x-www-form-urlencoded",
        "Accept": "text/plain",
        "User-agent": "python-requests/" + version,
    }

    # Encoding the request as a json string
    requestString = json.dumps(request)
    requestString = urlencode(requestString)

    # opening the https connection
    conn = httplib.HTTPSConnection(server)

    # Making the query
    conn.request("POST", "/api/v0/invoke", "request=" + requestString, headers)

    # Getting the response
    resp = conn.getresponse()
    head = resp.getheaders()
    content = resp.read().decode("utf-8")

    # Close the https connection
    conn.close()
    return head, content

In [0]:

import numpy as np
import requests
import pandas as pd
from IPython.display import display, HTML
import sys
import os
import time
import re
import json
from urllib.parse import quote as urlencode
from urllib.request import urlretrieve
import http.client as httpcli
from astropy.table import Table

planeturl = "https://exo.mast.stsci.edu/api/v0.1/exoplanets/"
dvurl = "https://exo.mast.stsci.edu/api/v0.1/dvdata/tess/"
header_secinfo = {}
header = {}


def aqTicReq(tessid, column):

    """Astroquery request from TESS candidates database"""

    requestTic = {
        "service": "Mast.Catalogs.Filtered.Tic",
        "format": "json",
        "params": {
            "columns": "c.{}".format(str(column)),  # Column name to get
            "filters": [
                {
                    "paramName": "ID",  # Filter by ID (tessid/ticid)
                    "values": [str(tessid)],
                }
            ],
        },
    }

    headers, outString = mastQuery(requestTic)
    outData = json.loads(outString)  # This isn't entirely necessary here
    return outData['data'][0][str(column)]


# Planet name assembler


def nameResolver(ticid):

    """Returns all name info of star. Can
    match to names from most major stellar
    catalogues"""

    request = {
        "service": "Mast.Name.Lookup",
        "params": {"input": str(ticid), "format": "json"},
    }

    headers, outString = mastQuery(request)

    outData = json.loads(outString)

    return outData


def pnameGenerator(tceid):

    """Planet name generator using the standard NASA
    exoplanet archive format. Used to generate likely
    names for as yet unnamed planets"""

    tessid, pl_number = tceid.split("-")
    tessid = str(int(tessid))
    pl_dict = {
        "01": "a",
        "02": "b",
        "03": "c",
        "04": "d",
        "05": "e",
        "06": "f",
        "07": "g",
        "08": "h",
        "09": "i",
        "10": "j",
        "11": "k",
        "12": "l",
    }
    star_name = nameResolver(tessid)["resolvedCoordinate"][0]["canonicalName"]
    return str(star_name + "-" + pl_dict[pl_number])


# exoMAST info getters: sector getter


def getSector(tessid, get1=True, tce="TCE_1"):

    """Get TESS sector information for threshold crossing event.
    Used to get light curves from exoMAST archive"""

    name = "TIC " + str(tessid)
    url = planeturl + "/identifiers/"
    params = {"name": name}
    r = requests.get(url=url, params=params, headers=header_secinfo)
    planet_names = r.json()
    # tce = planet_names['tessTCE']
    url = dvurl + str(tessid) + "/tces/"
    myparams = {"tce": tce}

    r = requests.get(url=url, params=myparams, headers=header_secinfo)
    sectorInfo = r.json()

    sectors = [x[:11] for x in sectorInfo["TCE"] if tce in x][0][:5]
    sector_split = [x.split(":") for x in sectorInfo["TCE"] if tce in x][0][:5]
    if get1 == True:
        return sectors
    else:
        return sector_split


def tceConverter(tce):

    """Convert <-xx> style TCE designation to <TCE_x> format"""

    tce_dict = {
        "01": "TCE_1",
        "02": "TCE_2",
        "03": "TCE_3",
        "04": "TCE_4",
        "05": "TCE_5",
        "06": "TCE_6",
        "07": "TCE_7",
        "08": "TCE_8",
        "09": "TCE_9",
        "10": "TCE_10",
        "11": "TCE_11",
        "12": "TCE_12",
    }
    return tce_dict[tce]


def getPlanetProp(tceid, prop_name):

    """Get single planet property from exoMAST.
    Takes tceid in <ticid>-<tce> format like: 
    00101365484-01. 11 digit ticid, <-xx> tce
    number"""

    tceid = str(tceid)
    tessid, pl_number = tceid.split("-")
    tce = tceConverter(pl_number)

    tessid = str(int(tessid))

    star_name = "TIC " + str(tessid)

    sectors = getSector(tessid, False, tce)
    full_pname = star_name + " (" + sectors[0].upper() + ") " + tce

    url = planeturl + str(full_pname) + "/properties/"
    r = requests.get(url=url, headers=header)
    planet_prop = r.json()

    if planet_prop[0][prop_name] == None:
        return_prop = 0
    else:
        return_prop = planet_prop[0][prop_name]
    
    return return_prop

In [0]:
#from .getter import aqTicReq, getSector, getPlanetProp, pnameGenerator
import pandas as pd


def getStarCsv(starlist, csv_out_name, verbose=1):
    """Gets star properties for list of stars. Outputs
    to specified csv file. Do not include .csv extension"""

    Stars = pd.DataFrame(
        columns=[
            "star_tessid",
            "twomass",
            "gaiaid",
            "sector",
            "magnitude",
            "luminosity",
            "star_mass",
            "star_radius",
            "constellation",
            #"disposition",
            "rightascension",
            "declination",
            #"metallicity"
        ]
    )

    i = 0
    for tessid in starlist:
        if tessid not in Stars["star_tessid"].values:
            if verbose == 1:
                print(".", end="")
            elif verbose >= 2:
                print(i, tessid)
            else:
                None
            star = {
                "star_tessid": int(tessid),
                "twomass": aqTicReq(tessid, "TWOMASS"),
                "gaiaid": aqTicReq(tessid, "GAIA"),
                "sector": getSector(tessid, True),
                "magnitude": aqTicReq(tessid, "Tmag"),
                "luminosity": aqTicReq(tessid, "lumclass"),
                "star_mass": aqTicReq(tessid, "mass"),
                "star_radius": aqTicReq(tessid, "rad"),
                "constellation": getPlanetProp(
                    str(f"{tessid:011}") + "-01", "constellation"
                ),
#                 "disposition": aqTicReq(tessid, "disposition"),
                "rightascension": str(round(float(aqTicReq(tessid, "ra")), 8)),
                "declination": str(round(float(aqTicReq(tessid, "dec")), 8)),
            }
            Stars = Stars.append(star, ignore_index=True)
            i += 1
            if verbose >= 3:
                print(star)
            Stars.to_csv(r"{}.csv".format(csv_out_name))

In [0]:
id = list(tess_id['ticid'])

In [0]:
getStarCsv(id,'star3')

..........................................................

In [0]:
star = pd.read_csv('star.csv')
star

,Unnamed: 0,star_tessid,twomass,gaiaid,sector,magnitude,luminosity,star_mass,star_radius,constellation,disposition,rightascension,declination,metallicity
0,0,139250626,23083495-4300530,6.541748e+18,s0001,8.3020,GIANT,NaN,33.070400,Grus,NaN,347.145616,-43.014701,NaN
1,1,139281803,23102280-4221542,6.542153e+18,s0001,8.6609,DWARF,1.050000,1.030850,Grus,NaN,347.595016,-42.365066,NaN
2,2,139285666,23115205-4508106,6.527965e+18,s0001,8.1197,DWARF,0.980000,0.854082,Grus,NaN,347.966890,-45.136287,NaN
3,3,139285669,23115362-4508004,6.527965e+18,s0002,11.2678,DWARF,0.524345,0.527264,Grus,NaN,347.973493,-45.133454,NaN
4,4,139300891,23125058-4445067,6.529478e+18,s0001,9.9010,DWARF,NaN,0.981901,Grus,NaN,348.210875,-44.751917,NaN
5,5,139370701,04290442-6936439,4.655916e+18,s0001,7.7873,DWARF,1.390000,1.311100,Hydrus,NaN,67.268452,-69.612211,NaN
6,6,140609384,04452817-7416500,4.652288e+18,s0001,9.8838,DWARF,1.080000,1.336870,Mensa,NaN,71.367414,-74.280572,NaN
7,7,140659980,04495359-7227454,4.653000e+18,s0005,8.9849,DWARF,1.000000,1.971280,Mensa,NaN,72.473353,-72.462619,NaN
8,8,140859822,05042740-7401505,NaN,s0001,12.0680,DWARF,0.211000,0.234000,Mensa,NaN,76.114400,-74.030600,NaN
9,9,140940493,05080432-7242324,4.651398e+18,s0005,10.2112,DWARF,0.970000,0.820647,Mensa,NaN,77.018036,-72.708989,NaN


In [0]:
del star['disposition']
del star['metallicity']

In [0]:
star

,Unnamed: 0,star_tessid,twomass,gaiaid,sector,magnitude,luminosity,star_mass,star_radius,constellation,rightascension,declination
0,0,139250626,23083495-4300530,6.541748e+18,s0001,8.3020,GIANT,NaN,33.070400,Grus,347.145616,-43.014701
1,1,139281803,23102280-4221542,6.542153e+18,s0001,8.6609,DWARF,1.050000,1.030850,Grus,347.595016,-42.365066
2,2,139285666,23115205-4508106,6.527965e+18,s0001,8.1197,DWARF,0.980000,0.854082,Grus,347.966890,-45.136287
3,3,139285669,23115362-4508004,6.527965e+18,s0002,11.2678,DWARF,0.524345,0.527264,Grus,347.973493,-45.133454
4,4,139300891,23125058-4445067,6.529478e+18,s0001,9.9010,DWARF,NaN,0.981901,Grus,348.210875,-44.751917
5,5,139370701,04290442-6936439,4.655916e+18,s0001,7.7873,DWARF,1.390000,1.311100,Hydrus,67.268452,-69.612211
6,6,140609384,04452817-7416500,4.652288e+18,s0001,9.8838,DWARF,1.080000,1.336870,Mensa,71.367414,-74.280572
7,7,140659980,04495359-7227454,4.653000e+18,s0005,8.9849,DWARF,1.000000,1.971280,Mensa,72.473353,-72.462619
8,8,140859822,05042740-7401505,NaN,s0001,12.0680,DWARF,0.211000,0.234000,Mensa,76.114400,-74.030600
9,9,140940493,05080432-7242324,4.651398e+18,s0005,10.2112,DWARF,0.970000,0.820647,Mensa,77.018036,-72.708989


In [0]:
del star['Unnamed: 0']
star

,star_tessid,twomass,gaiaid,sector,magnitude,luminosity,star_mass,star_radius,constellation,rightascension,declination
0,139250626,23083495-4300530,6.541748e+18,s0001,8.3020,GIANT,NaN,33.070400,Grus,347.145616,-43.014701
1,139281803,23102280-4221542,6.542153e+18,s0001,8.6609,DWARF,1.050000,1.030850,Grus,347.595016,-42.365066
2,139285666,23115205-4508106,6.527965e+18,s0001,8.1197,DWARF,0.980000,0.854082,Grus,347.966890,-45.136287
3,139285669,23115362-4508004,6.527965e+18,s0002,11.2678,DWARF,0.524345,0.527264,Grus,347.973493,-45.133454
4,139300891,23125058-4445067,6.529478e+18,s0001,9.9010,DWARF,NaN,0.981901,Grus,348.210875,-44.751917
5,139370701,04290442-6936439,4.655916e+18,s0001,7.7873,DWARF,1.390000,1.311100,Hydrus,67.268452,-69.612211
6,140609384,04452817-7416500,4.652288e+18,s0001,9.8838,DWARF,1.080000,1.336870,Mensa,71.367414,-74.280572
7,140659980,04495359-7227454,4.653000e+18,s0005,8.9849,DWARF,1.000000,1.971280,Mensa,72.473353,-72.462619
8,140859822,05042740-7401505,NaN,s0001,12.0680,DWARF,0.211000,0.234000,Mensa,76.114400,-74.030600
9,140940493,05080432-7242324,4.651398e+18,s0005,10.2112,DWARF,0.970000,0.820647,Mensa,77.018036,-72.708989


In [0]:
star3 = pd.read_csv('star3.csv')

In [0]:
star3

,Unnamed: 0,star_tessid,twomass,gaiaid,sector,magnitude,luminosity,star_mass,star_radius,constellation,disposition,rightascension,declination
0,0,139250626,23083495-4300530,6.541748e+18,s0001,8.3020,GIANT,NaN,33.070400,Grus,NaN,347.145616,-43.014701
1,1,139281803,23102280-4221542,6.542153e+18,s0001,8.6609,DWARF,1.050000,1.030850,Grus,NaN,347.595016,-42.365066
2,2,139285666,23115205-4508106,6.527965e+18,s0001,8.1197,DWARF,0.980000,0.854082,Grus,NaN,347.966890,-45.136287
3,3,139285669,23115362-4508004,6.527965e+18,s0002,11.2678,DWARF,0.524345,0.527264,Grus,NaN,347.973493,-45.133454
4,4,139300891,23125058-4445067,6.529478e+18,s0001,9.9010,DWARF,NaN,0.981901,Grus,NaN,348.210875,-44.751917
5,5,139370701,04290442-6936439,4.655916e+18,s0001,7.7873,DWARF,1.390000,1.311100,Hydrus,NaN,67.268452,-69.612211
6,6,140609384,04452817-7416500,4.652288e+18,s0001,9.8838,DWARF,1.080000,1.336870,Mensa,NaN,71.367414,-74.280572
7,7,140659980,04495359-7227454,4.653000e+18,s0005,8.9849,DWARF,1.000000,1.971280,Mensa,NaN,72.473353,-72.462619
8,8,140859822,05042740-7401505,NaN,s0001,12.0680,DWARF,0.211000,0.234000,Mensa,NaN,76.114400,-74.030600
9,9,140940493,05080432-7242324,4.651398e+18,s0005,10.2112,DWARF,0.970000,0.820647,Mensa,NaN,77.018036,-72.708989


In [0]:
tess_id

,Unnamed: 0,tceid,ticid,planetNumber,planetCandidateCount,starTeffKelvin,starLoggCgs,orbitalPeriodDays,orbitalPeriodDays_err,planetRadiusEarthRadii,planetRadiusEarthRadii_err,ratioPlanetRadiusToStarRadius,ratioSemiMajorAxisToStarRadius,semiMajorAxisAu,starRadiusSolarRadii,transitDepthPpm,transitDepthPpm_err,transitDurationHours,transitEpochBtjd,transitEpochBtjd_err,transitIngressTimeHours,minImpactParameter,ws_mes,ws_mesphase,expectedtransitcount,chiSquare2,mes,maxses,equilibriumTempKelvin,InsolationFlux
0,0,00139250626-01,139250626,1,1,3752.00,4.43800,31.331800,0.002674,3.13843,1.124100,0.028748,65.72550,0.194601,1.000000,988.228,119.2040,3.746600,1347.45,0.001899,0.104707,0.003412,6.18694,1.79583,2,4.27020,8.60686,7.67340,375.303,4.688670
1,1,00139281803-01,139281803,1,1,5818.00,4.42453,30.004600,0.011727,4.57590,0.649222,0.040315,18.95560,0.192042,1.039700,1252.430,172.6290,4.975790,1348.62,0.010050,1.960100,0.955717,3.95842,-6.50833,2,1.64843,8.61389,7.42655,597.340,30.089100
2,2,00139285666-01,139285666,1,2,5561.91,4.43800,0.698056,0.000030,1.30396,0.319271,0.013904,1.78311,0.013924,0.859049,200.875,28.3924,2.472830,1325.47,0.001075,0.070341,0.699984,10.50500,0.35000,81,85.90260,7.67025,4.59272,1927.720,3263.610000
3,3,00139285666-02,139285666,2,2,5561.91,4.43800,0.697913,0.000028,1.27712,0.614572,0.013618,2.18318,0.013922,0.859049,213.246,26.1335,2.426400,1325.83,0.000986,0.040160,0.363446,2.82927,0.49633,80,60.81050,8.90267,3.59044,1927.850,3264.510000
4,4,00139285669-01,139285669,1,1,3260.00,4.43800,0.697970,0.000120,22.16320,17.902200,0.203016,1.28088,0.015407,1.000000,19604.200,1637.0100,5.427770,1325.47,0.004998,2.713880,0.969172,11.91440,0.34444,81,59.90600,10.46970,3.74261,1158.920,426.317000
5,5,00139300891-01,139300891,1,2,5380.00,4.42414,0.909372,0.000058,2.01278,2.050910,0.018777,1.82858,0.017964,0.981901,366.693,50.0413,3.152760,1325.46,0.001795,0.119273,0.699495,7.09093,0.25694,62,35.97930,8.15177,3.00604,1755.110,2242.520000
6,6,00139300891-02,139300891,2,2,5380.00,4.42414,0.454922,0.000020,1.98714,2.491050,0.018538,2.16689,0.011321,0.981901,393.200,39.7448,1.567410,1325.67,0.001017,0.037000,0.423618,3.89878,0.12010,123,181.26600,10.11510,5.42536,2210.920,5646.890000
7,7,00139370701-01,139370701,1,1,6632.00,4.43800,1.120060,0.000066,2.12472,0.360944,0.012857,1.91393,0.027841,1.513710,169.743,22.0864,3.554540,1325.60,0.001650,0.096602,0.716097,6.46343,0.74167,50,27.49040,8.39120,2.93959,2157.840,5123.850000
8,8,00140609384-01,140609384,1,1,5925.50,4.26541,21.603100,0.004042,6.46235,0.874399,0.046738,16.79170,0.155733,1.266530,1767.610,190.5460,4.448030,1327.80,0.004725,1.579090,0.944214,2.84735,-0.36806,3,3.49859,13.31310,11.63160,745.650,73.056700
9,9,00140659980-01,140659980,1,2,5642.40,3.00000,9.440230,0.000055,74.64960,13.706800,0.327094,15.50660,0.047422,2.090510,44672.000,44071.7000,4.068230,1328.27,0.000158,2.034120,1.000000,261.59300,4.45833,6,3506.18000,520.39100,262.36900,1653.080,1764.780000


In [0]:
merged = tess_id.merge(star, left_on='ticid', right_on='star_tessid')

In [0]:
merged.head()

,Unnamed: 0,tceid,ticid,planetNumber,planetCandidateCount,starTeffKelvin,starLoggCgs,orbitalPeriodDays,orbitalPeriodDays_err,planetRadiusEarthRadii,planetRadiusEarthRadii_err,ratioPlanetRadiusToStarRadius,ratioSemiMajorAxisToStarRadius,semiMajorAxisAu,starRadiusSolarRadii,transitDepthPpm,transitDepthPpm_err,transitDurationHours,transitEpochBtjd,transitEpochBtjd_err,transitIngressTimeHours,minImpactParameter,ws_mes,ws_mesphase,expectedtransitcount,chiSquare2,mes,maxses,equilibriumTempKelvin,InsolationFlux,star_tessid,twomass,gaiaid,sector,magnitude,luminosity,star_mass,star_radius,constellation,rightascension,declination
0,0,00139250626-01,139250626,1,1,3752.00,4.43800,31.331800,0.002674,3.13843,1.124100,0.028748,65.72550,0.194601,1.000000,988.228,119.2040,3.74660,1347.45,0.001899,0.104707,0.003412,6.18694,1.79583,2,4.27020,8.60686,7.67340,375.303,4.68867,139250626,23083495-4300530,6.541748e+18,s0001,8.3020,GIANT,NaN,33.070400,Grus,347.145616,-43.014701
1,1,00139281803-01,139281803,1,1,5818.00,4.42453,30.004600,0.011727,4.57590,0.649222,0.040315,18.95560,0.192042,1.039700,1252.430,172.6290,4.97579,1348.62,0.010050,1.960100,0.955717,3.95842,-6.50833,2,1.64843,8.61389,7.42655,597.340,30.08910,139281803,23102280-4221542,6.542153e+18,s0001,8.6609,DWARF,1.050000,1.030850,Grus,347.595016,-42.365066
2,2,00139285666-01,139285666,1,2,5561.91,4.43800,0.698056,0.000030,1.30396,0.319271,0.013904,1.78311,0.013924,0.859049,200.875,28.3924,2.47283,1325.47,0.001075,0.070341,0.699984,10.50500,0.35000,81,85.90260,7.67025,4.59272,1927.720,3263.61000,139285666,23115205-4508106,6.527965e+18,s0001,8.1197,DWARF,0.980000,0.854082,Grus,347.966890,-45.136287
3,3,00139285666-02,139285666,2,2,5561.91,4.43800,0.697913,0.000028,1.27712,0.614572,0.013618,2.18318,0.013922,0.859049,213.246,26.1335,2.42640,1325.83,0.000986,0.040160,0.363446,2.82927,0.49633,80,60.81050,8.90267,3.59044,1927.850,3264.51000,139285666,23115205-4508106,6.527965e+18,s0001,8.1197,DWARF,0.980000,0.854082,Grus,347.966890,-45.136287
4,4,00139285669-01,139285669,1,1,3260.00,4.43800,0.697970,0.000120,22.16320,17.902200,0.203016,1.28088,0.015407,1.000000,19604.200,1637.0100,5.42777,1325.47,0.004998,2.713880,0.969172,11.91440,0.34444,81,59.90600,10.46970,3.74261,1158.920,426.31700,139285669,23115362-4508004,6.527965e+18,s0002,11.2678,DWARF,0.524345,0.527264,Grus,347.973493,-45.133454


In [0]:
del merged['Unnamed: 0']

In [0]:
merged.to_csv('merged.csv')